In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# dataset_red=pd.read_csv("Data/winequality-red.csv",sep=";",index_col=False)
# dataset_white=pd.read_csv("Data/winequality-white.csv",sep=";",index_col=False)
dataset_red=pd.read_csv("C:/Users/10253/Desktop/Machine-Learning-Project-Group-6/Data/winequality-red.csv",sep=";",index_col=False)
dataset_white=pd.read_csv("C:/Users/10253/Desktop/Machine-Learning-Project-Group-6/Data/winequality-white.csv",sep=";",index_col=False)

red=["red"]*len(dataset_red)
dataset_red.insert(0,"Label",red)
white=["white"]*len(dataset_white)
dataset_white.insert(0,"Label",white)
# dataset_white.head(n=5)
dataset=pd.concat([dataset_red,dataset_white],axis=0)
dataset.set_index("Label")

X=dataset.drop("Label",axis=1)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
Xs=scaler.fit_transform(X)


# from sklearn.preprocessing import LabelEncoder
# le=LabelEncoder()
# dataset["Label"]=le.fit_transform(dataset["Label"])
y=dataset["Label"]


from sklearn.feature_selection import SelectKBest,f_classif
Xs=SelectKBest(f_classif,k=4).fit_transform(Xs,y)


from sklearn.model_selection import train_test_split, cross_val_score

Xs_train,Xs_test,y_train,y_test=train_test_split(Xs,y,test_size=0.3,random_state=1,stratify=y)



In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf_fs_cv=DecisionTreeClassifier(criterion="entropy")



p_grid={"splitter":["best","random"],"max_features":[1,2,3,4],"max_depth":[2,3,4,5,6,7,8]}


inner_cv=KFold(n_splits=3,shuffle=True)
outer_cv=KFold(n_splits=5,shuffle=True)


clf = GridSearchCV(estimator=clf_fs_cv, param_grid=p_grid, cv=inner_cv)
nested_score = cross_val_score(clf, X=Xs, y=y, cv=outer_cv)

print(nested_score.mean())
print(y)

clf_fs_cv.fit(Xs_train,y_train)
confusion_matrix(clf_fs_cv.predict(Xs_test),y_test)

0.9832853395229634
0       0
1       0
2       0
3       0
4       0
       ..
4893    1
4894    1
4895    1
4896    1
4897    1
Name: Label, Length: 6497, dtype: int32


array([[ 463,   14],
       [  17, 1456]], dtype=int64)